<a href="https://colab.research.google.com/github/Kambojharsh/Tweet-Support-Classifier/blob/main/BTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 9.4 MB/s eta 0:00:00


In [ ]:
import re

corpus=open("/content/drive/MyDrive/Deep Learning/words.txt").read().splitlines()

In [ ]:
import pandas as pd
import random as rd
import re
import emoji
import math
import string
import numpy as np
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df= pd.read_csv('/content/drive/MyDrive/Deep Learning/annotations - Final_Dataset.csv')
df = df[df['Reply_Class'] != 'Lang']

In [ ]:
hashtag_cache = {}

def separate_hashtag_words(sentence):
    # Check if the result is cached
    sentence = sentence.lower()
    if sentence in hashtag_cache:
        return hashtag_cache[sentence]

    # Calculate and cache the result
    hashtags = re.findall(r'#(\w+)', sentence.lower())
    store = []
    for hashtag in hashtags:
        list_words = []
        words = re.split(r'(\d+)', hashtag)
        def func():
            for word in words:
                list_words.append(word)
            return list_words
        word_list = func()
        for element in word_list:
            if element.isdigit():
                store.append(element)
            else:
                new_word = element
                word_separated, val1, val2 = '', 0, 0
                while val2 < 30:
                    for each_word in range(len(new_word), 0, -1):
                        if new_word[val1:each_word] in corpus:
                            word_separated = word_separated + ' ' + new_word[val1:each_word]
                            val1 = each_word
                            break
                    val2 += 1
                store.append(word_separated.strip())

    # Format the output string
    output_string = ' '.join(str(x) for x in store).title()
    if output_string.endswith(","):
        output_string = output_string[:-1]  # Remove the trailing comma
    # Cache the result
    hashtag_cache[sentence] = output_string
    return output_string.lower()

In [ ]:

output_string = separate_hashtag_words("#sanctionsonpakistan")
print(output_string)

sanctions on pakistan


# Preprocessing


In [ ]:


# def pre_process_tweet(tweet):
#     # Remove \n from the end of the sentence
#     tweet = tweet.strip('\n')

#     # Use regex to remove '@' only if followed by a word
#     tweet = re.sub(r'@(\w+)', r'\1', tweet)

#     # Remove any URL
#     tweet = re.sub(r"http\S+", "", tweet)
#     tweet = re.sub(r"www\S+", "", tweet)

#     # Convert emojis to their underscore-separated representation
#     tweet = emoji.demojize(tweet)
#     tweet = re.sub(r':(\w+):', r'\1 ', tweet)

#     # Convert underscore-separated words to space-separated words
#     tweet = re.sub(r'_', ' ', tweet)

#     # Convert dash-separated words to space-separated words
#     tweet = re.sub(r'-', ' ', tweet)

#     # Replace &amp with 'and'
#     tweet = tweet.replace('&amp','and')
#     tweet = tweet.replace('&AMP','and')
#     tweet = tweet.replace('â','')

#     # Replace U.S. with 'usa'
#     tweet = tweet.replace('U.S.', 'usa')
#     tweet = tweet.replace('US','usa')

#     # Remove colons from the end of the sentences (if any)
#     tweet = tweet.strip()
#     if tweet.endswith(':'):
#         tweet = tweet[:-1]

#     # Remove hash-tags symbols
#     tweet = tweet.replace('#', '')

#     # Convert every word to lowercase
#     tweet = tweet.lower()

#     # Remove punctuations
#     tweet = tweet.translate(str.maketrans('', '', string.punctuation))

#     # Trim extra spaces
#     tweet = " ".join(tweet.split())

#     return tweet




# # Example usage:
# tweet = "😙😚Here is an 'example' tweet! @user #example PM's us US #sanctionpakistan http://example.com  😂💀💀do a lot !!"
# processed_tweet = pre_process_tweet(tweet)
# print(processed_tweet)


kissing face with smiling eyes kissing face with closed eyes here is an example tweet user example pms us usa sanctionpakistan face with tears of joy skull skull do a lot


In [ ]:

def pre_process_tweet(tweet):
    # Remove \n from the end of the sentence
    tweet = tweet.strip('\n')

    # Convert emojis to their descriptions
    tweet = emoji.demojize(tweet)
    # Add spaces between emoji descriptions
    tweet = re.sub(r'(:\w+:)', lambda x: x.group(0).replace(':', ' '), tweet)

    # Remove any URL
    tweet = re.sub(r"http\S+", "", tweet)
    tweet = re.sub(r"www\S+", "", tweet)

    # Convert underscore-separated words to space-separated words
    tweet = re.sub(r'_', ' ', tweet)

    # Convert dash-separated words to space-separated words
    tweet = re.sub(r'-', ' ', tweet)

    # Replace &amp with 'and'
    tweet = tweet.replace('&amp','and')
    tweet = tweet.replace('&AMP','and')

    # Replace U.S. with 'usa'
    tweet = tweet.replace('U.S.', 'usa')
    tweet = tweet.replace('US','usa')
    # Remove colons from the end of the sentences (if any)
    tweet = tweet.strip()
    if tweet.endswith(':'):
        tweet = tweet[:-1]

    # Split tweet into words
    # words = tweet.split()

    # # Iterate over words and replace hashtag words
    # for i, word in enumerate(words):
    #     if word.startswith('#'):
    #         words[i] = separate_hashtag_words(word)

    # tweet = ' '.join(words)
    # tweet = re.sub(r'#\w+', lambda x: separate_hashtag_words(x.group()), tweet)
    # Remove hash-tags symbols and add spaces between words
    # tweet = re.sub(r'#(\w+)', r' \1', tweet)

    # Convert every word to lowercase
    tweet = tweet.lower()
    tweet = tweet.replace('#sanctionpakistan','sanction pakistan ')
    tweet = tweet.replace('#unitednations','united nations ')
    tweet = tweet.replace('#endproxywar','end proxy war ')
    tweet = tweet.replace('#saveafghanistan','save afghanistan ')
    tweet = tweet.replace('#blamegameonpakistan','blame game on pakistan ')
    tweet = tweet.replace('#afghanlivesmatter','afghan lives matter ')
    tweet = tweet.replace('#pakistaniwarcrimes','pakistani war crimes ')
    tweet = tweet.replace('#endproxywarinafghanistan','end proxy war in afghanistan ')

    tweet = tweet.replace('#', '')
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    # Trim extra spaces
    tweet = " ".join(tweet.split())

    return tweet

# Example usage:
tweet = "#sanctionpakistan"
processed_tweet = pre_process_tweet(tweet)
print(processed_tweet)
tweet = "😙😚Here is an 'example' tweet! @user #example PM's us US  #sanctionpakistan http://example.com  😂💀💀do a lot !!"
processed_tweet = pre_process_tweet(tweet)
print(processed_tweet)



sanction pakistan
kissing face with smiling eyes kissing face with closed eyes here is an example tweet user example pms us usa sanction pakistan face with tears of joy skull skull do a lot


In [ ]:
df['Text'] = df['Text'].apply(pre_process_tweet)
df['Reply'] = df['Reply'].apply(pre_process_tweet)

In [ ]:
print(df['Reply'][30])

sanction pakistan


# Tokenization and Padding


In [ ]:
# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Text'] + df['Reply'])

max_sequence_length = 60

sequences_text = tokenizer.texts_to_sequences(df['Text'])
sequences_reply = tokenizer.texts_to_sequences(df['Reply'])

padded_sequences_text = pad_sequences(sequences_text, maxlen=max_sequence_length, padding='post')
padded_sequences_reply = pad_sequences(sequences_reply, maxlen=max_sequence_length, padding='post')



reply_tokens = [tokenizer.index_word.get(token, '') for token in padded_sequences_reply[11]]
print("Reply tokens:", reply_tokens)
print(padded_sequences_reply[11])

Reply tokens: ['sanction', 'pakistan', 'united', 'nations', 'united', 'nations', 'high', 'commissioner', 'for', 'human', 'rights', 'united', 'nations', 'general', 'assemblyinternational', 'court', 'of', 'justice', 'international', 'courts', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
[  57   18  361  760  361  760 1302 4357   11  518  599  361  760  855
 4358 2408    4  500  479 3029    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


# Glove dictionary creation

In [ ]:

words = dict()

def add_to_dict(d, filename):
  with open(filename, 'r') as f:
    for line in f.readlines():
      line = line.split(' ')

      try:
        d[line[0]] = np.array(line[1:], dtype=float)
      except:
        continue

add_to_dict(words, '/content/drive/MyDrive/Deep Learning/glove.twitter.27B.50d.txt')
words

#Create Embeddings

In [ ]:


# Function to get embeddings for sequences
def get_embeddings(sequences, word_dict):
    embedding_dim = 50
    # embedding_dim = len(next(iter(word_dict.values())))  # Dimension of GloVe embeddings
    num_words = len(tokenizer.word_index) + 1  # Adding 1 for padding token (index 0)

    # Initialize embeddings matrix
    embeddings_matrix = np.zeros((len(sequences), max_sequence_length, embedding_dim))

    # Iterate over sequences
    for i, seq in enumerate(sequences):
        # Iterate over tokens in each sequence
        for j, token_index in enumerate(seq):
            if token_index != 0:  # Skip padding token
                word = tokenizer.index_word.get(token_index)
                if word in word_dict:
                    embeddings_matrix[i, j, :] = word_dict[word]
                else:
                    # If word not in GloVe dictionary, use zero vector
                    embeddings_matrix[i, j, :] = np.zeros(embedding_dim)

    return embeddings_matrix

# Get embeddings for padded_sequences_text and padded_sequences_reply
embeddings_text = get_embeddings(padded_sequences_text, words)
embeddings_reply = get_embeddings(padded_sequences_reply, words)

# Print the shape of embeddings for verification
# print("Shape of embeddings_text:", embeddings_text.shape)
# print("Shape of embeddings_reply:", embeddings_reply.shape)

print(embeddings_reply[30][1]) #1st word of 1st tweet text


[ 0.88084   0.30409   0.52879  -1.0601   -0.61752   0.36665  -1.0673
  0.58799   0.30702  -0.39185  -0.1807   -2.1897   -2.9295    0.085642
  0.46994  -0.66581  -0.81312  -0.28203   0.59902   0.063795 -0.49994
 -0.015774 -0.44039   0.38011  -0.064951  0.60016  -0.41667  -0.20307
  0.14874   0.6782    1.0253   -0.51524   0.11897   0.3691    1.8187
 -0.35305  -0.22244   0.77367   0.16845  -1.8854   -0.09891  -0.39681
  0.064795  0.23272  -0.16004   0.33406   0.012795 -0.84146   0.71641
  0.40234 ]


# CREATION OF TRAIN, TEST, VAL

In [ ]:
y=df['Reply_Class']


In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split

# Assuming you have your data stored in embeddings_text and embeddings_reply arrays
# Split the data into training (80%), validation (10%), and test (10%) sets
X_train_text, X_temp_text = train_test_split(embeddings_text, test_size=0.2, random_state=42)
X_val_text, X_test_text = train_test_split(X_temp_text, test_size=0.5, random_state=42)

X_train_reply, X_temp_reply = train_test_split(embeddings_reply, test_size=0.2, random_state=42)
X_val_reply, X_test_reply = train_test_split(X_temp_reply, test_size=0.5, random_state=42)

# Assuming you have labels stored in a variable y
# Split the labels accordingly
y_train, y_temp = train_test_split(y, test_size=0.2, random_state=42)
y_val, y_test = train_test_split(y_temp, test_size=0.5, random_state=42)

# Convert the arrays to numpy arrays
X_train_text = np.array(X_train_text)
X_val_text = np.array(X_val_text)
X_test_text = np.array(X_test_text)

X_train_reply = np.array(X_train_reply)
X_val_reply = np.array(X_val_reply)
X_test_reply = np.array(X_test_reply)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# Print the shapes to verify
print("X_train_text shape:", X_train_text.shape)
print("X_val_text shape:", X_val_text.shape)
print("X_test_text shape:", X_test_text.shape)
print("X_train_reply shape:", X_train_reply.shape)
print("X_val_reply shape:", X_val_reply.shape)
print("X_test_reply shape:", X_test_reply.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)


X_train_text shape: (2291, 60, 50)
X_val_text shape: (286, 60, 50)
X_test_text shape: (287, 60, 50)
X_train_reply shape: (2291, 60, 50)
X_val_reply shape: (286, 60, 50)
X_test_reply shape: (287, 60, 50)
y_train shape: (2291,)
y_val shape: (286,)
y_test shape: (287,)


In [ ]:
from tensorflow.keras import layers
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM

model = Sequential([])

model.add(layers.Input(shape=(60, 50)))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 64)            29440     
                                                                 
 dropout (Dropout)           (None, 60, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 64)            33024     
                                                                 
 dropout_1 (Dropout)         (None, 60, 64)            0         
                                                                 
 lstm_2 (LSTM)               (None, 60, 64)            33024     
                                                                 
 dropout_2 (Dropout)         (None, 60, 64)            0         
                                                                 
 flatten (Flatten)           (None, 3840)              0

#LSTM MODEL

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM

# Define the input embedding (example embedding of a single sentence)
embedding = embeddings_reply[0] # Example embedding of shape (60, 50)

# Define the Sequential model
model = Sequential()

# Add the LSTM layer to the model
model.add(LSTM(64, input_shape=(60, 50)))

# Compile the model (not necessary for this demonstration)
model.compile(optimizer='adam', loss='mse')  # Any dummy optimizer and loss for demonstration purpose

# Print the model summary
print(model.summary())

# Predict the intermediate output for the input embedding
intermediate_output = model.predict(np.expand_dims(embedding, axis=0))

# Print the shape of the intermediate output
print("Shape of intermediate output:", intermediate_output.shape)

# Print the intermediate output itself
print("Intermediate output:")
print(intermediate_output)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
Total params: 29440 (115.00 KB)
Trainable params: 29440 (115.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
1/1 [==============================] - 0s 401ms/step
Shape of intermediate output: (1, 64)
Intermediate output:
[[ 6.86205531e-05 -6.04103889e-06  8.04972879e-05 -2.70733854e-05
   5.62488785e-05 -4.27173181e-05 -1.26177227e-04 -4.11624678e-05
  -7.58803726e-05  1.77580168e-05  4.34321373e-05  2.31117247e-05
  -9.52698683e-05 -6.64584804e-05  2.39157962e-05 -8.82857858e-05
   8.96582278e-05  2.58536766e-05 -3.29308568e-05  1.40438806e-05
  -7.84855365e-05 -5.29215395e-05 -1.11455192e-05  4.62953685e-05
  -8.30017743e-05 -1.96008637e-04